In [1]:
import os
import time
import numpy as np
import pandas as pd
pd.set_option('precision', 4)

import torch
import torch.optim as optim

from tools.args_tools import args, print_dict

args.max_epochs = 30
args.lr_scheduler = True
args.clip = True
args.clip_max_norm = 500
args.batch_norm = True
args.input_with_grid = True
args.lr_scheduler = True
args.clip = True
args.batch_norm = True

print_dict(args)

working_folder: /home/utmostof9/ssd/01_ty_research
radar_folder: /home/utmostof9/ssd/01_ty_research/01_radar_data
radar_wrangled_data_folder: /home/utmostof9/ssd/01_ty_research/01_radar_data/02_wrangled_files
meteorology_folder: /home/utmostof9/ssd/01_ty_research/02_meteorological_data
meteorology_wrangled_folder: /home/utmostof9/ssd/01_ty_research/02_meteorological_data/01_wrangled_files
ty_info_folder: /home/utmostof9/ssd/01_ty_research/03_ty_info
ty_info_wrangled_folder: /home/utmostof9/ssd/01_ty_research/03_ty_info/01_wrangled_files
ty_list: /home/utmostof9/ssd/01_ty_research/ty_list.csv
result_folder: /home/utmostof9/ssd/01_ty_research/04_results
params_folder: /home/utmostof9/ssd/01_ty_research/05_params
able_cuda: True
gpu: 0
value_dtype: torch.float32
max_epochs: 30
batch_size: 8
loss_function: <function BMSE at 0x7fb87f51b268>
optimizer: <class 'torch.optim.adam.Adam'>
lr: 0.0001
lr_scheduler: True
weight_decay: 0.1
clip: True
clip_max_norm: 500
batch_norm: True
normalize_targ

In [2]:
from tools.trajGRU import model
from tools.run import train, test, get_dataloader

In [3]:
args.device

device(type='cuda', index=0)

### Dataloader

In [4]:
trainloader, testloader = get_dataloader(args)
for i in trainloader:
    print(i['input'].shape)

torch.Size([8, 6, 9, 90, 90])


KeyboardInterrupt: 

### Construct learning model

In [4]:
inputs_channels = 1+len(args.meteorology_list)
# set the factor of cnn channels
c = args.channel_factor

## construct Traj GRU
# initialize the parameters of the encoders and forecasters

rnn_link_size = [13, 13, 9]

encoder_input_channel = inputs_channels
encoder_downsample_channels = [2*c,32*c,96*c]
encoder_rnn_channels = [32*c,96*c,96*c]

forecaster_input_channel = 0
forecaster_upsample_channels = [96*c,96*c,4*c]
forecaster_rnn_channels = [96*c,96*c,32*c]

if args.I_shape[0] == args.F_shape[0]*3:
    encoder_downsample_k = [5,4,3]
    encoder_downsample_s = [3,2,2]
    encoder_downsample_p = [1,1,1]
elif args.I_shape[0] == args.F_shape[0]:
    encoder_downsample_k = [3,4,4]
    encoder_downsample_s = [1,2,2]
    encoder_downsample_p = [1,1,1]

encoder_rnn_k = [3,3,3]
encoder_rnn_s = [1,1,1]
encoder_rnn_p = [1,1,1]
encoder_n_layers = 6

forecaster_upsample_k = [3,4,3]
forecaster_upsample_s = [2,2,1]
forecaster_upsample_p = [1,1,1]

forecaster_rnn_k = [3,3,3]
forecaster_rnn_s = [1,1,1]
forecaster_rnn_p = [1,1,1]
forecaster_n_layers = 6


forecaster_output = 1
forecaster_output_k = 3
forecaster_output_s = 1
forecaster_output_p = 1
forecaster_output_layers = 1

Net = model(n_encoders=args.input_frames, n_forecasters=args.output_frames, rnn_link_size=rnn_link_size, 
            encoder_input_channel=encoder_input_channel, encoder_downsample_channels=encoder_downsample_channels,
            encoder_rnn_channels=encoder_rnn_channels, encoder_downsample_k=encoder_downsample_k,
            encoder_downsample_s=encoder_downsample_s, encoder_downsample_p=encoder_downsample_p, 
            encoder_rnn_k=encoder_rnn_k,encoder_rnn_s=encoder_rnn_s, encoder_rnn_p=encoder_rnn_p, 
            encoder_n_layers=encoder_n_layers, forecaster_input_channel=forecaster_input_channel, 
            forecaster_upsample_channels=forecaster_upsample_channels, forecaster_rnn_channels=forecaster_rnn_channels,
            forecaster_upsample_k=forecaster_upsample_k, forecaster_upsample_s=forecaster_upsample_s, 
            forecaster_upsample_p=forecaster_upsample_p, forecaster_rnn_k=forecaster_rnn_k, forecaster_rnn_s=forecaster_rnn_s,
            forecaster_rnn_p=forecaster_rnn_p, forecaster_n_layers=forecaster_n_layers, forecaster_output=forecaster_output, 
            forecaster_output_k=forecaster_output_k, forecaster_output_s=forecaster_output_s, 
            forecaster_output_p=forecaster_output_p, forecaster_output_layers=forecaster_output_layers, 
            batch_norm=args.batch_norm).to(args.device, dtype=args.value_dtype)

### Training process

In [5]:
time_s = time.time()
train(net=Net, trainloader=trainloader, testloader=testloader, args=args)
time_e = time.time()
t = time_e-time_s
h = int((t)//3600)
m = int((t-h*3600)//60)
s = int(t-h*3600-m*60)

print('The total computing time of this training process: {:d}:{:d}:{:d}'.format(h,m,s))

lr: 1.0e-04


NameError: name 'device' is not defined